In [178]:
import numpy as np
import pandas as pd
from difflib import get_close_matches
from tqdm import tqdm_notebook

This is the [original dataset](https://wheresthejump.com/full-movie-list/). There are a bunch of movies which have the same name but were made by different directors in different years. I am removing those entries since I have to join this table with a few others and the other tables do not have the director information. 

In [179]:
df_agg = pd.read_csv('jump_scare_df.csv')
df_agg['netflix_us'] = df_agg['netflix_us'] == 'Yes'
df_agg['year'] = pd.to_datetime(df_agg['year'].astype('str'), format='%Y')
df_agg.drop(['netflix_us','url'], axis=1, inplace=True)
df_agg.drop_duplicates('movie_name', keep='last', inplace=True)
print(df_agg.shape)
df_agg.head()

(527, 6)


,movie_name,director,year,jump_count,jump_scare_rating,imdb
0,10 Cloverfield Lane,Dan Trachtenberg,2016-01-01,8,2.5,7.3
1,28 Days Later,Danny Boyle,2003-01-01,10,3.0,7.6
2,28 Weeks Later,Juan Carlos Fresnadillo,2007-01-01,17,3.5,7.0
3,30 Days of Night,David Slade,2007-01-01,9,2.5,6.6
4,31,Rob Zombie,2016-01-01,8,2.5,5.8


In [180]:
df_runtime = pd.read_json('movieListExtended.json').T.reset_index(drop=True)
df_runtime.drop(['link','synopsis','mpaaRating','rottenTomatoesRating','tags','srt'], axis=1, inplace=True)
df_runtime.drop_duplicates('title', keep='last', inplace=True)
print(df_runtime.shape)
df_runtime.head()

(527, 2)


,title,runtime
0,10 Cloverfield Lane,103
1,28 Days Later,113
2,28 Weeks Later,100
3,30 Days of Night,113
4,31,103


In [181]:
df_agg = pd.merge(df_agg, df_runtime, left_on='movie_name', right_on='title', how='inner').drop('title', axis=1)
print(df_agg.shape)
df_agg.head()

(526, 7)


,movie_name,director,year,jump_count,jump_scare_rating,imdb,runtime
0,10 Cloverfield Lane,Dan Trachtenberg,2016-01-01,8,2.5,7.3,103
1,28 Days Later,Danny Boyle,2003-01-01,10,3.0,7.6,113
2,28 Weeks Later,Juan Carlos Fresnadillo,2007-01-01,17,3.5,7.0,100
3,30 Days of Night,David Slade,2007-01-01,9,2.5,6.6,113
4,31,Rob Zombie,2016-01-01,8,2.5,5.8,103


Next I get a dataset of timestamps of all jumpscares in the movies. It is in a non-tabular JSON format so I use an [online convertor](https://codebeautify.org/jsonviewer) to change it into a long-form table and carry out the rest of the cleaning in pandas.

In [182]:
df = pd.read_csv('scare_timestamps.csv', header=None).T
df.columns = ['A', 'B']
print(df.shape)
df.head()

(9351, 2)


,A,B
0,10-cloverfield-lane-2016.srt.0.ts,00:05:13.000
1,10-cloverfield-lane-2016.srt.0.type,major
2,10-cloverfield-lane-2016.srt.1.ts,00:19:58.000
3,10-cloverfield-lane-2016.srt.1.type,minor
4,10-cloverfield-lane-2016.srt.2.ts,00:38:15.000


In [183]:
d = pd.DataFrame(df['A'].str.split('.').tolist(), columns=['Name','dummy','scare_num', 'var'])
d.head()

,Name,dummy,scare_num,var
0,10-cloverfield-lane-2016,srt,0,ts
1,10-cloverfield-lane-2016,srt,0,type
2,10-cloverfield-lane-2016,srt,1,ts
3,10-cloverfield-lane-2016,srt,1,type
4,10-cloverfield-lane-2016,srt,2,ts


In [184]:
df = pd.concat([d, df['B']], axis=1)
df.head()

,Name,dummy,scare_num,var,B
0,10-cloverfield-lane-2016,srt,0,ts,00:05:13.000
1,10-cloverfield-lane-2016,srt,0,type,major
2,10-cloverfield-lane-2016,srt,1,ts,00:19:58.000
3,10-cloverfield-lane-2016,srt,1,type,minor
4,10-cloverfield-lane-2016,srt,2,ts,00:38:15.000


In [185]:
d2 = pd.DataFrame(df['Name'].str.rsplit('-', 1).tolist(), columns=['name','year'])
d2.head()

,name,year
0,10-cloverfield-lane,2016
1,10-cloverfield-lane,2016
2,10-cloverfield-lane,2016
3,10-cloverfield-lane,2016
4,10-cloverfield-lane,2016


In [186]:
df = pd.concat([d2, df[['scare_num','var','B']]], axis=1)
df.head()

,name,year,scare_num,var,B
0,10-cloverfield-lane,2016,0,ts,00:05:13.000
1,10-cloverfield-lane,2016,0,type,major
2,10-cloverfield-lane,2016,1,ts,00:19:58.000
3,10-cloverfield-lane,2016,1,type,minor
4,10-cloverfield-lane,2016,2,ts,00:38:15.000


In [187]:
df = df.pivot_table(index=['name','year','scare_num'], columns='var', values='B', aggfunc='first').reset_index()
df.columns.name = ''
df

,name,year,scare_num,ts,type
0,10-cloverfield-lane,2016,0,00:05:13.000,major
1,10-cloverfield-lane,2016,1,00:19:58.000,minor
2,10-cloverfield-lane,2016,2,00:38:15.000,minor
3,10-cloverfield-lane,2016,3,00:41:43.000,minor
4,10-cloverfield-lane,2016,4,01:15:08.000,minor
...,...,...,...,...,...
4670,zombieland,2009,4,00:13:05.000,minor
4671,zombieland,2009,5,00:16:27.000,major
4672,zombieland,2009,6,00:17:55.000,minor
4673,zombieland,2009,7,01:15:03.000,minor


In [188]:
new_names = []
for name in tqdm_notebook(df['name']):
    try:
        x = get_close_matches(name, df_agg['movie_name'], cutoff=0.7)[0]
        new_names.append(x)
    except:
        new_names.append('')

In [189]:
df['clean_name'] = new_names
df.head()

,name,year,scare_num,ts,type,clean_name
0,10-cloverfield-lane,2016,0,00:05:13.000,major,10 Cloverfield Lane
1,10-cloverfield-lane,2016,1,00:19:58.000,minor,10 Cloverfield Lane
2,10-cloverfield-lane,2016,2,00:38:15.000,minor,10 Cloverfield Lane
3,10-cloverfield-lane,2016,3,00:41:43.000,minor,10 Cloverfield Lane
4,10-cloverfield-lane,2016,4,01:15:08.000,minor,10 Cloverfield Lane


This movie has a missing year in the timestamp dataset so I am manually fixing it. 

In [190]:
df.loc[df['name'].str.contains('summer-of'), 'name'] = 'summer-of-84'
df.loc[df['name'].str.contains('summer-of'), 'clean_name'] = 'Summer of 84'
df.loc[df['name'].str.contains('summer-of'), 'year'] = 2018

In [191]:
df['year'] = pd.to_datetime(df['year'].astype('str'), format='%Y')
df['ts_delta'] = pd.to_timedelta(df['ts']).dt.seconds
df['ts'] = pd.to_datetime(df['ts'], format='%H:%M:%S.%f')
print(df.shape)
df.head()

(4675, 7)


,name,year,scare_num,ts,type,clean_name,ts_delta
0,10-cloverfield-lane,2016-01-01,0,1900-01-01 00:05:13,major,10 Cloverfield Lane,313
1,10-cloverfield-lane,2016-01-01,1,1900-01-01 00:19:58,minor,10 Cloverfield Lane,1198
2,10-cloverfield-lane,2016-01-01,2,1900-01-01 00:38:15,minor,10 Cloverfield Lane,2295
3,10-cloverfield-lane,2016-01-01,3,1900-01-01 00:41:43,minor,10 Cloverfield Lane,2503
4,10-cloverfield-lane,2016-01-01,4,1900-01-01 01:15:08,minor,10 Cloverfield Lane,4508


Some of these missing movies are not present in the original csv but others have been miss-matched during the string comparison part. 

In [192]:
df_agg['movie_name'].nunique(), df['clean_name'].nunique()

(526, 329)

The missing movies are dropped automatically during the merge, then I drop the mis-matched ones.

In [193]:
df_ts = pd.merge(df, df_agg, left_on='clean_name', right_on='movie_name', how='inner')
df_ts.drop((df_ts[df_ts['year_x']!=df_ts['year_y']]).index, axis=0, inplace=True)
df_ts.drop(['year_y','clean_name'], axis=1, inplace=True)
df_ts.rename({'year_x':'year'}, axis=1, inplace=True)
df_ts['runtime'] = df_ts['runtime'] * 60.0
df_ts.head()

,name,year,scare_num,ts,type,ts_delta,movie_name,director,jump_count,jump_scare_rating,imdb,runtime
0,10-cloverfield-lane,2016-01-01,0,1900-01-01 00:05:13,major,313,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180
1,10-cloverfield-lane,2016-01-01,1,1900-01-01 00:19:58,minor,1198,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180
2,10-cloverfield-lane,2016-01-01,2,1900-01-01 00:38:15,minor,2295,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180
3,10-cloverfield-lane,2016-01-01,3,1900-01-01 00:41:43,minor,2503,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180
4,10-cloverfield-lane,2016-01-01,4,1900-01-01 01:15:08,minor,4508,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180


Two movies have jumpscares which have a timestamp which exceeds the movie's runtime. I'm removing those movies. 

In [194]:
df_ts = df_ts[~df_ts['name'].isin(df_ts[df_ts['ts_delta'] > df_ts['runtime']]['name'])]

Now I calculate the relative posiiton of jumpscares in each movie. This essentially normalizes the positions for longer and shorter movies. 

In [195]:
df_ts['position'] = df_ts['ts_delta'] / df_ts['runtime']

In [196]:
df_ts.drop('name', axis=1, inplace=True)
print(df_ts.shape)
df_ts.head()

(3077, 12)


,year,scare_num,ts,type,ts_delta,movie_name,director,jump_count,jump_scare_rating,imdb,runtime,position
0,2016-01-01,0,1900-01-01 00:05:13,major,313,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180,0.0506472
1,2016-01-01,1,1900-01-01 00:19:58,minor,1198,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180,0.193851
2,2016-01-01,2,1900-01-01 00:38:15,minor,2295,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180,0.371359
3,2016-01-01,3,1900-01-01 00:41:43,minor,2503,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180,0.405016
4,2016-01-01,4,1900-01-01 01:15:08,minor,4508,10 Cloverfield Lane,Dan Trachtenberg,8,2.5,7.3,6180,0.72945


I'm also going to remove the the movies before 1950. These movies were made so far before the rest of the dataset that there can be no logical pattern between them and the far off movies mess with the scales on plots. 

In [202]:
df_ts = df_ts[df_ts['year'] > '1950']
df_agg = df_agg[df_agg['year'] > '1950']

Saving as pickle to maintain the column datatypes. 

In [203]:
df_ts.to_pickle('data_w_timestamps.pkl')
df_agg.to_pickle('data_original.pkl')